In [2]:
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt 
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.animation as anim
%matplotlib qt

torch.set_default_dtype(torch.float64)

### Model Definition

In [14]:
class MLP(nn.Module):
    def __init__(self, num_features, num_classes):
        super(MLP, self).__init__()
        middle_layer_size = 20
        self.layers = nn.Sequential(
            nn.Linear(num_features, middle_layer_size),
            nn.ReLU(),
            nn.Linear(middle_layer_size, num_features),
        )
        self.output = nn.Sequential(
            nn.Linear(num_features, num_classes),
#             nn.Sigmoid()
        )
        print(f"Number of featues: {num_features}, number of classes: {num_classes}")
        
    def forward(self, x):
        return self.output(self.layers(x))
    
    def data_transformation(self, x):
        return self.layers(x)
    
    def forward_from_transformation(self, z):
        return self.output(z)

### Dataset Creation

In [4]:
def generate_dataset(t: np.ndarray, num_classes: int, class_index: int):
    sigma = 0.2 + 0.01 * class_index
    mean = 0.0
    random_number = 0
    return t * np.array([
        np.sin(2 * np.pi / num_classes * (2 * t + class_index - 1)  + np.random.normal(0, sigma, t.shape)),
        np.cos(2 * np.pi / num_classes * (2 * t + class_index - 1) + np.random.normal(0, sigma, t.shape)),
    ])

In [5]:
t = np.linspace(0, 1, 5000)
num_classes = 3
train_data = [None] * num_classes
for i in range(num_classes):
    print(num_classes)
    data_class = generate_dataset(t, num_classes, i + 1).T
    plt.scatter(data_class[:, 0], data_class[:, 1], s = 1.2)
    
    train_data[i] = np.append(data_class, i * np.ones((data_class.shape[0], 1)), axis = 1)
    
plt.show()

3
3
3


In [6]:
train_data = np.concatenate(train_data)
train_X, train_y = train_data[:, :2], train_data[:, 2].astype(int)
train_X.shape, train_y.shape

((15000, 2), (15000,))

In [7]:
y = np.zeros((train_y.size, train_y.max() + 1))
y[np.arange(train_y.size), train_y] = 1
y.shape, y

((15000, 3),
 array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]]))

In [8]:
X, y = torch.Tensor(train_X), torch.Tensor(y)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, shuffle = True, batch_size = 512)

### Training

In [15]:
model = MLP(2, num_classes)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()

Number of featues: 2, number of classes: 3


In [16]:
epochs = 100
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

def plot_decision_boundary():
    x_min, x_max = train_X[:, 0].min() - 1, train_X[:, 0].max() + 1
    y_min, y_max = train_X[:, 1].min() - 1, train_X[:, 1].max() + 1
    h = 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    ax.clear()
    Z = model(torch.Tensor(np.c_[xx.ravel(), yy.ravel()])).argmax(1)

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z)
    ax.axis('off')
    # Plot also the training points
    ax.scatter(X[:, 0], X[:, 1], c = y, s = 1.2)
    ax.set_title('Learning of Non linear decision boundary by NN')

def train(epoch):
    running_loss = 0
    last_loss = 0
    for i, data in enumerate(dataloader):
        inp, labels = data
        optimizer.zero_grad()

        outputs = model(inp)
        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()
        running_loss += loss.item()
    plot_decision_boundary()
    if epoch % 5 == 0:
        print(f'Epoch {epoch} loss: {running_loss / i}')
    running_loss = 0.
a = anim.FuncAnimation(fig, train, frames=epochs, repeat = False)
plt.show()

Epoch 0 loss: 1.2039089033554107
Epoch 0 loss: 1.0867703258674484
Epoch 5 loss: 0.902660964015247
Epoch 10 loss: 0.7402182261479422
Epoch 15 loss: 0.6458164298579173
Epoch 20 loss: 0.4939487620478414
Epoch 25 loss: 0.36501558052885263
Epoch 30 loss: 0.2888144146325956
Epoch 35 loss: 0.24242961838092647
Epoch 40 loss: 0.21097331852243606
Epoch 45 loss: 0.1866312040059497
Epoch 50 loss: 0.16733498265293384
Epoch 55 loss: 0.15215934126311192
Epoch 60 loss: 0.13841374471357168
Epoch 65 loss: 0.12862809514718082
Epoch 70 loss: 0.12078585642583398
Epoch 75 loss: 0.11365307444382558
Epoch 80 loss: 0.10872326578064781
Epoch 85 loss: 0.10352127491516594
Epoch 90 loss: 0.09863643008526995
Epoch 95 loss: 0.09426334382651283


In [ ]:
a.save("neural_network.gif")

Epoch 0 loss: 1.1315163163908393
Epoch 0 loss: 1.1252764690000527
Epoch 0 loss: 1.1182216822335462
Epoch 5 loss: 1.0427523904825338
Epoch 10 loss: 0.8791632925698722
Epoch 15 loss: 0.8073771079736877


In [ ]:
# transform data
transformed_X = model.data_transformation(torch.Tensor(train_X))
transformed_X = transformed_X.detach().cpu().numpy()
x_min, x_max = transformed_X[:, 0].min() - 1, transformed_X[:, 0].max() + 1
y_min, y_max = transformed_X[:, 1].min() - 1, transformed_X[:, 1].max() + 1
h = 0.01
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
# plt.rcParams["figure.figsize"] = (8,8)
fig, ax = plt.subplots(2, 1, figsize = (10, 10))

ax[0].scatter(data_class_1[:, 0], data_class_1[:, 1], s = 1.2)
ax[0].scatter(data_class_2[:, 0], data_class_2[:, 1], s = 1.2)
ax[0].scatter(data_class_3[:, 0], data_class_3[:, 1], s = 1.2)

Z = model.forward_from_transformation(torch.Tensor(np.c_[xx.ravel(), yy.ravel()])).argmax(1)
print(xx.shape, yy.shape, Z.shape)
Z = Z.reshape(xx.shape)
ax[1].contourf(xx, yy, Z)

transformed_data = model.data_transformation(torch.tensor(data_class_1)).detach().cpu().numpy()
ax[1].scatter(transformed_data[:, 0], transformed_data[:, 1], s = 1.2)

transformed_data = model.data_transformation(torch.tensor(data_class_2)).detach().cpu().numpy()
ax[1].scatter(transformed_data[:, 0], transformed_data[:, 1], s = 1.2)

transformed_data = model.data_transformation(torch.tensor(data_class_3)).detach().cpu().numpy()
ax[1].scatter(transformed_data[:, 0], transformed_data[:, 1], s = 1.2)


plt.title("Transformation of data points from input space by Neural network")
plt.savefig("Neural Network Data transformation Visualization.png")
plt.show()

Here, in the above plot we can see that the data is linearyly seperable in the space transformed by neural networks.

In [ ]:
x_min, x_max = train_X[:, 0].min() - 1, train_X[:, 0].max() + 1
y_min, y_max = train_X[:, 1].min() - 1, train_X[:, 1].max() + 1
h = 0.01
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
fig, ax = plt.subplots()
Z = model(torch.Tensor(np.c_[xx.ravel(), yy.ravel()])).argmax(1)

# Put the result into a color plot
print(xx.shape, Z.shape)
Z = Z.reshape(xx.shape)
ax.contourf(xx, yy, Z)
ax.axis('off')

# Plot also the training points
ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, s = 1.2)

ax.set_title('Non linear decision boundary')

In [ ]:
plt.close('all')